# Binary Search

In [ ]:
code = """
#include<mpi.h>
#include<stdio.h>

#define n 12

#define key 55

int a[] = {1,2,3,4,7,9,13,24,55,56,67,88};

int a2[20];

int binarySearch(int *array, int start, int end, int value) {
    int mid;
    
    while(start <= end) {
        mid = (start + end) / 2;
        if(array[mid] == value) 
            return mid;
        else if(array[mid] > value)
            end = mid - 1;
        else
            start = mid + 1;
    }
    return -1;
}

int main(int argc, char* argv[]) {
    int pid, np, elements_per_process, n_elements_received;
    
    MPI_Status status;
    
    MPI_Init(&argc, &argv);
    
    MPI_Comm_rank(MPI_COMM_WORLD, &pid);
    MPI_Comm_size(MPI_COMM_WORLD, &np);

    //printf("np = %d", np);
    
    if(pid == 0) {
        int index, i;
        
        if(np > 1) {
            for(i=1; i<np-1; i++) {
                
                index = i * elements_per_process;
                //element count
                MPI_Send(&elements_per_process, 1, MPI_INT, i, 0, MPI_COMM_WORLD);

                MPI_Send(&a[index], elements_per_process, MPI_INT, i, 0, MPI_COMM_WORLD);
            
            }
            
            index = i* elements_per_process;
            
            int elements_left = n - index;
            
            MPI_Send(&elements_left, 1, MPI_INT, i, 0, MPI_COMM_WORLD);
            MPI_Send(&a[index], elements_left, MPI_INT, i, 0, MPI_COMM_WORLD);
        }
        
        int position = binarySearch(a, 0, elements_per_process-1, key);
        
        if(position != -1)
          printf("Found at: %d position", position);
        
        int temp;
        
        for(i=1; i<np; i++) {
            MPI_Recv(&temp, 1, MPI_INT, MPI_ANY_SOURCE, 0, MPI_COMM_WORLD, &status);
            int sender = status.MPI_SOURCE;
            
            if(temp != -1)
                printf("Found at: %d position by %d process number", (sender*elements_per_process)+temp, sender);
        }
    }
    
    else {
        MPI_Recv(&n_elements_received, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &status);
        
        MPI_Recv(&a2, n_elements_received, MPI_INT, 0, 0, MPI_COMM_WORLD, &status);
    
        int position = binarySearch(a2, 0, n_elements_received-1, key);
        
        MPI_Send(&position, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
    }
    
    MPI_Finalize();
    
    return 0;
}
"""

In [ ]:
file_ = open("BinarySearch.c", "w");
file_.write(code);
file_.close();

In [ ]:
!mpiCC BinarySearch.c

In [ ]:
!mpirun --allow-run-as-root -np 4 ./a.out

Found at: 8 position by 3 process number

# Best First Search

In [ ]:
code = """
#include <bits/stdc++.h>
#include <omp.h>

using namespace std;

list<int>q;   // queue
vector<int>w(7,1000);   // weight 
int visited[7];   // visited array

// Comp function to select best node
struct Comp {
  bool operator ()(const int & a, const int & b) {
    return w[a]<w[b];
  }
};

// best first search function
void bestFirstSearch(int adj_matrix[7][7], int n_nodes) {
  // if q is empty, return
  if(q.empty())
    return;

  // sort queue such that node with lowest weight is in front
  q.sort(Comp());

  // pop front of queue
  int cur_node = q.front();
  q.pop_front();
  printf("%d, ", cur_node);
  
  #pragma omp parallel for shared(visited)
  for(int i=0; i<n_nodes; i++) {
    if(adj_matrix[cur_node][i] > 0 && visited[i] == 0) {
      if(w[i] > adj_matrix[cur_node][i]){
        w[i] = adj_matrix[cur_node][i];
      }
      q.push_back(i);
      visited[i]=1;
    }
  }
  
  bestFirstSearch(adj_matrix, n_nodes);
}
int main() {
  int th = omp_get_max_threads();   // get max threads
  cout << "Max Threads : " << th << endl;

  int n_nodes = 7;    // total nodes = 7

  // mark all nodes as not visited
  for(int i=0; i<n_nodes; i++) {    
    visited[i] = 0;
  }

  // weight matrix
  int adj_matrix[7][7] = {
                            {0,   10,   20,   0,    0,    0,    0},
                            {10,   0,   30,  30,    0,    0,    0},
                            {20,  30,    0,   0,   40,    0,    0},
                            {0,   30,    0,   0,   20,    0,    0},
                            {0,    0,   40,  20,    0,   10,    0},
                            {0,    0,    0,   0,   10,    0,   20},
                            {0,    0,    0,   0,    0,   20,    0}
                        };

  int start_node = 3;   // start node

  q.push_back(start_node);    // push start node in queue
  
  w[start_node] = 0;    // weight of start node is 0
  
  visited[start_node] = 1;    // mark start node as visited
  
  bestFirstSearch(adj_matrix, n_nodes);   // call best first search function
  
  return 0;
}
"""

In [ ]:
file_ = open("bfs.cpp", "w")
file_.write(code)
file_.close()

In [ ]:
!g++ -fopenmp bfs.cpp

In [ ]:
!./a.out

Max Threads : 2
3, 4, 5, 6, 1, 0, 2, 